In [9]:
from gklearn.utils.graphfiles import loadDataset
import networkx as nx
import matplotlib
matplotlib.use('TkAgg')

import matplotlib.pyplot as plt
import numpy as np
from __future__ import print_function
import torch
import sys
#sys.path.append('C:/Users/Nandan/Desktop/BIonet Proj/deep-ged-main/deepged')
import GPUtil
from tqdm import tqdm
from scipy.optimize import linear_sum_assignment
from scipy.sparse import csr_matrix
import torch.nn.init as init


#from deepged import optim

In [10]:
def label_to_color(label):
    if label == 'C':
        return 0.1
    elif label == 'O':
        return 0.8
    
def nodes_to_color_sequence(G):
    return [label_to_color(c[1]['label'][0]) for c in G.nodes(data=True)]
Gs,y = loadDataset('C:/Users/z004u7ws/Desktop/Bionet/deep-ged-main/data/Acyclic/trainset_1.ds')
#for e in Gs[13].edges():
#    print(Gs[13][e[0]][e[1]]['bond_type'])
print('edge max label',max(max([[G[e[0]][e[1]]['bond_type'] for e in G.edges()] for G in Gs])))
G1 = Gs[1]
G2 = Gs[9]
print(y[13],y[23])
#nx.draw_networkx(G1,with_labels=True,node_color = nodes_to_color_sequence(G1),cmap='autumn')
#plt.figure()

#nx.draw_networkx(G2,with_labels=True,node_color = nodes_to_color_sequence(G2),cmap='autumn')




from itertools import chain


def compute_star(G, id_node, label_node, label_edge):
    '''
    Calcule une string contenant les labels des noeuds voisins plus le label du noeud central.
    '''
    central_label = G.nodes(data=True)[id_node][label_node]
    neighs = []
    for id_neigh, labels_e in G[id_node].items():
        neigh_label = G.nodes(data=True)[id_neigh][label_node]
        extended_label = ''.join([labels_e[label_edge], neigh_label[0]])
        neighs.append(extended_label)
    neigh_labels = ''.join(sorted(neighs, key=str))
    new_label = ''.join([central_label[0], '_', neigh_labels])
    return new_label


def compute_extended_labels(G, label_node='atom', label_edge='bond_type'):
    '''
    Calcule l'ensemble des labels étendus pour un graphe G.
    Rajoute le nouveau label au graphe G
    '''
    for v in G.nodes():
        new_label = compute_star(G, v, label_node, label_edge)
        G.nodes[v]['extended_label'] = new_label
    return G


def extract_edge_labels(list_of_graphs, edge_label="bond_type"):
    '''
    Calcul l'ensemble des labels d'aretes pour tout les graphes dans list_of_graphs
    '''
    return set(chain(*[[e[2][edge_label] for e in a_graph.edges(
        data=True)] for a_graph in list_of_graphs]))


def extract_node_labels(list_of_graphs, node_label="atom_type"):
    '''
    Calcul l'ensemble des labels de noeuds pour tout les graphes dans list_of_graphs
    '''
    return set(chain(*[[v[1][node_label] for v in a_graph.nodes(
        data=True)] for a_graph in list_of_graphs]))


def build_node_dictionnary(list_of_graphs,
                           node_label="atom_type", edge_label="bond_type"):
  
    ensemble_labels = extract_node_labels(list_of_graphs, node_label)
    dict_labels = {k: v for k, v in zip(
        sorted(ensemble_labels), range(len(ensemble_labels)))}
    edge_labels = extract_edge_labels(list_of_graphs, edge_label)
    nb_edge_labels = len(edge_labels)
    return dict_labels, nb_edge_labels


for g in Gs:
    print(compute_extended_labels(g))
for v in Gs[10].nodes():
        print(Gs[10].nodes[v])

edge max label 1
64.4 52.5
Graph with 3 nodes and 2 edges
Graph with 3 nodes and 2 edges
Graph with 4 nodes and 3 edges
Graph with 4 nodes and 3 edges
Graph with 5 nodes and 4 edges
Graph with 5 nodes and 4 edges
Graph with 4 nodes and 3 edges
Graph with 5 nodes and 4 edges
Graph with 5 nodes and 4 edges
Graph with 5 nodes and 4 edges
Graph with 5 nodes and 4 edges
Graph with 6 nodes and 5 edges
Graph with 6 nodes and 5 edges
Graph with 6 nodes and 5 edges
Graph with 6 nodes and 5 edges
Graph with 5 nodes and 4 edges
Graph with 5 nodes and 4 edges
Graph with 5 nodes and 4 edges
Graph with 6 nodes and 5 edges
Graph with 6 nodes and 5 edges
Graph with 6 nodes and 5 edges
Graph with 6 nodes and 5 edges
Graph with 6 nodes and 5 edges
Graph with 6 nodes and 5 edges
Graph with 6 nodes and 5 edges
Graph with 6 nodes and 5 edges
Graph with 6 nodes and 5 edges
Graph with 7 nodes and 6 edges
Graph with 7 nodes and 6 edges
Graph with 7 nodes and 6 edges
Graph with 7 nodes and 6 edges
Graph with 6

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#torch.autograd.set_detect_anomaly(True)

class Net(nn.Module):
    
        
    def __init__(self,GraphList,normalize=False,node_label='label'):
        super(Net, self).__init__()   
        self.normalize=normalize
        self.node_label=node_label
        dict,self.nb_edge_labels=self.build_node_dictionnary(GraphList)
        self.nb_labels=len(dict)
        print('nb labels:',self.nb_labels,' ; nb_edge_labels:',self.nb_edge_labels)
        self.device=torch.device("cpu")
        nb_node_pair_label=self.nb_labels*(self.nb_labels-1)/2.0
        nb_edge_pair_label=int(self.nb_edge_labels*(self.nb_edge_labels-1)/2)
        self.node_weighs=nn.Parameter(torch.tensor(1.0/(nb_node_pair_label+nb_edge_pair_label+2))+(1e-3)*torch.rand(int(nb_node_pair_label+1),requires_grad=True,device=self.device)) # all substitution costs+ nodeIns/Del. old version: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del        
        self.edge_weighs=nn.Parameter(torch.tensor(1.0/(nb_node_pair_label+nb_edge_pair_label+2))+(1e-3)*torch.rand(nb_edge_pair_label+1,requires_grad=True,device=self.device)) #edgeIns/Del
        
        #self.coef_dist=nn.Parameter(torch.tensor(1.0,requires_grad=True,device=self.device))
        
        self.card=torch.tensor([G.order() for G in GraphList]).to(self.device)
        card_max=self.card.max()
        self.A=torch.empty((len(GraphList),card_max*card_max),dtype=torch.int,device=self.device)
        self.labels=torch.empty((len(GraphList),card_max),dtype=torch.int,device=self.device)
        print(self.A.shape)
        for k in range(len(GraphList)):
            A,l=self.from_networkx_to_tensor(GraphList[k],dict)             
            self.A[k,0:A.shape[1]]=A[0]
            self.labels[k,0:l.shape[0]]=l
        print('adjacency matrices',self.A)
        print('node labels',self.labels)
        print('order of the graphs',self.card)
        
    def forward(self, input):        
        ged=torch.zeros(len(input)).to(self.device) 
        node_costs,nodeInsDel,edge_costs,edgeInsDel=self.from_weighs_to_costs()
       
        
        #print('weighs:',self.weighs.device,'device:',self.device,'card:',self.card.device,'A:',self.A.device,'labels:',self.labels.device)
        for k in range(len(input)):            
            g1=input[k][0]
            g2=input[0][1]
            n=self.card[g1]
            m=self.card[g2]
            C=self.construct_cost_matrix(g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)      
            S = self.from_cost_to_similarity(C) 
            Xsh, iterations = self.sinkhorn_d1d2(S)
           
            row_ind, col_ind = linear_sum_assignment(Xsh.detach().numpy().max()-Xsh.detach().numpy())
            ged[k] = C[row_ind, col_ind].sum()
            
        return ged
       
            
    
    def from_weighs_to_costs(self):
        
        #cn=torch.exp(self.node_weighs)
        #ce=torch.exp(self.edge_weighs)
        cn=self.node_weighs*self.node_weighs
        ce=self.edge_weighs*self.edge_weighs
        total_cost=(cn.sum()+ce.sum())
        cn=cn/total_cost
        ce=ce/total_cost
        edgeInsDel=ce[-1]

        node_costs=torch.zeros((self.nb_labels,self.nb_labels),device=self.device)
        upper_part=torch.triu_indices(node_costs.shape[0],node_costs.shape[1],offset=1,device=self.device)        
        node_costs[upper_part[0],upper_part[1]]=cn[0:-1]
        node_costs=node_costs+node_costs.T
        if self.nb_edge_labels>1:
            edge_costs=torch.zeros((self.nb_edge_labels,self.nb_edge_labels),device=self.device)
            upper_part=torch.triu_indices(edge_costs.shape[0],edge_costs.shape[1],offset=1,device=self.device)        
            edge_costs[upper_part[0],upper_part[1]]=ce[0:-1]
            edge_costs=edge_costs+edge_costs.T
        else:
            edge_costs=torch.zeros(0,device=self.device)
        
        return node_costs,cn[-1],edge_costs,edgeInsDel
    
    def build_node_dictionnary(self,GraphList):
        #extraction de tous les labels d'atomes
        node_labels=[]
        for G in Gs:
            for v in nx.nodes(G):
                if not G.nodes[v][self.node_label][0] in node_labels:
                    node_labels.append(G.nodes[v][self.node_label][0])
        node_labels.sort()
        dict={}
        k=0
        for label in node_labels:
            dict[label]=k
            k=k+1
        print(node_labels)
        print(dict,len(dict))
    
        return dict,max(max([[int(G[e[0]][e[1]]['bond_type']) for e in G.edges()] for G in GraphList]))
    
    def from_networkx_to_tensor(self,G,dict):    
       # A=torch.tensor(nx.to_scipy_sparse_matrix(G,dtype=int,weight='bond_type').todense(),dtype=torch.int)        
       # lab=[dict[G.nodes[v][self.node_label][0]] for v in nx.nodes(G)]
        nodes = list(G.nodes())
    
    # Create an empty adjacency matrix with dtype=int
        num_nodes = len(nodes)
        A = torch.zeros((num_nodes, num_nodes), dtype=torch.int)
    
    # Iterate through edges and populate the adjacency matrix
        for edge in G.edges(data=True):
            source = nodes.index(edge[0])
            target = nodes.index(edge[1])
            bond_type = int(edge[2]['bond_type'])
            A[source][target] = bond_type
            A[target][source] = bond_type  # Assuming the graph is undirected
    
        # Convert node labels to a tensor
        lab = [dict[G.nodes[v][self.node_label][0]] for v in nodes]
        lab = torch.tensor(lab)
   
        return (A.view(1,A.shape[0]*A.shape[1]),torch.tensor(lab))

    def construct_cost_matrix(self,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel):
        n = self.card[g1].item()
        m = self.card[g2].item()
        
        A1=torch.zeros((n+1,n+1),dtype=torch.int,device=self.device)
        A1[0:n,0:n]=self.A[g1][0:n*n].view(n,n)
        A2=torch.zeros((m+1,m+1),dtype=torch.int,device=self.device)
        A2[0:m,0:m]=self.A[g2][0:m*m].view(m,m)
        
        
         # costs: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del
        
        #C=cost[3]*torch.cat([torch.cat([C12[l][k] for k in range(n+1)],1) for l in range(n+1)])
        #Pas bien sur mais cela semble fonctionner.
        C=edgeInsDel*self.matrix_edgeInsDel(A1,A2)
        if self.nb_edge_labels>1:
            for k in range(self.nb_edge_labels):
                for l in range(self.nb_edge_labels):
                    if k != l:
#                    C.add_(self.matrix_edgeSubst(A1,A2,k+1,l+1),alpha=edge_costs[k][l])
                        C=C+edge_costs[k][l]*self.matrix_edgeSubst(A1,A2,k+1,l+1)
        #C=cost[3]*torch.tensor(np.array([ [  k!=l and A1[k//(m+1),l//(m+1)]^A2[k%(m+1),l%(m+1)] for k in range((n+1)*(m+1))] for l in range((n+1)*(m+1))]),device=self.device)        

        l1=self.labels[g1][0:n]
        l2=self.labels[g2][0:m]
        D=torch.zeros((n+1)*(m+1),device=self.device)
        D[n*(m+1):]=nodeInsDel
        D[n*(m+1)+m]=0
        D[[i*(m+1)+m for i in range(n)]]=nodeInsDel
        D[[k for k in range(n*(m+1)) if k%(m+1) != m]]=torch.tensor([node_costs[l1[k//(m+1)],l2[k%(m+1)]] for k in range(n*(m+1)) if k%(m+1) != m],device=self.device )
        mask = torch.diag(torch.ones_like(D))
        C=mask*torch.diag(D) + (1. - mask)*C
        
        #C[range(len(C)),range(len(C))]=D
        #import ipdb; ipdb.set_trace()
        return C
    def matrix_edgeInsDel(self,A1,A2):
        Abin1=(A1!=torch.zeros((A1.shape[0],A1.shape[1]),device=self.device))
        Abin2=(A2!=torch.zeros((A2.shape[0],A2.shape[1]),device=self.device))
        C1=torch.einsum('ij,kl->ijkl',torch.logical_not(Abin1),Abin2)
        C2=torch.einsum('ij,kl->ijkl',Abin1,torch.logical_not(Abin2))
        C12=torch.logical_or(C1,C2).int()
    
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C12,1),1),0),1)

    def matrix_edgeSubst(self,A1,A2,lab1,lab2):
        Abin1=(A1==lab1*torch.ones((A1.shape[0],A1.shape[1]),device=self.device)).int()
        Abin2=(A2==lab2*torch.ones((A2.shape[0],A2.shape[1]),device=self.device)).int()
        C=torch.einsum('ij,kl->ijkl',Abin1,Abin2)
        
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C,1),1),0),1)
    
        
    def sinkhorn_d1d2(self, S, nb_iter=1500, eps=1e-3):
        n, m = S.shape
        ones_m = torch.ones(m, device=S.device)
        ones_n = torch.ones(n, device=S.device)
        c = ones_m
        converged = False
        i = 0
        while i <= nb_iter and not converged:
            rp = 1.0 / (torch.matmul(S, c))
            rp[-1] = 1.0
    
            if i >= 1:
                norm_r = torch.linalg.norm(r / rp - ones_n, ord=float('inf'))

            r = rp
            cp = 1.0 / (torch.matmul(S.T, r))
            cp[-1] = 1.0

            norm_c = torch.linalg.norm(c / cp - ones_m, ord=float('inf'))
            c = cp

            if i >= 1:
                converged = (norm_r <= eps) and (norm_c <= eps)

            i += 1

        # Return assignment matrix
        return torch.diag(r) @ S @ torch.diag(c), i

    def from_cost_to_similarity(self, C):
        M = torch.max(C) + 1.0
        S = 2 * M * torch.ones((C.shape[0], C.shape[1]))
        S[-1, :] = M
        S[:, -1] = M

        def simplify_matrix(S):
            def compute_threshold(S, n, m):
                ones_n = torch.ones((n, 1))
                ones_m = torch.ones((1, m))

                # chaque elt i,j a pour seuil S[n+1,j] + S[i,m+1]
                C = ones_n @ (S[-1, 0:m].reshape(1, m))
                C += S[0:n, -1].reshape(n, 1) @ ones_m
                return C

            return S - compute_threshold(S, S.shape[0], S.shape[1])

        return simplify_matrix(S)
    
      

In [17]:
""""class RegressionMLP(nn.Module):
    def __init__(self, input_size):
        super(RegressionMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 132)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return x """
        
        
class RegressionMLP(nn.Module):
    def __init__(self, input_size):
        super(RegressionMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 100)
        self.dropout = nn.Dropout(0.3)  # Adding dropout

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # Applying dropout
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

"""class RegressionMLP(nn.Module):
    def __init__(self, input_size):
        super(RegressionMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        
        # 1D Convolutional layer
        self.conv1d = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        
        self.fc2 = nn.Linear(16, 128)
        self.fc3 = nn.Linear(128, 100)
        self.dropout = nn.Dropout(0.3)  # Adding dropout

    def forward(self, x):
        # Apply fully connected layer
        x = torch.relu(self.fc1(x))
        
        # Reshape for 1D convolution
        x = x.unsqueeze(1)  # Add a channel dimension
        x = torch.relu(self.conv1d(x))
        x = x.view(x.size(0), -1)  # Flatten
        
        # Apply linear layers
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        
        # Apply dropout
        x = self.dropout(x)
        
        return x"""




'class RegressionMLP(nn.Module):\n    def __init__(self, input_size):\n        super(RegressionMLP, self).__init__()\n        self.fc1 = nn.Linear(input_size, 256)\n        \n        # 1D Convolutional layer\n        self.conv1d = nn.Conv1d(1, 16, kernel_size=3, padding=1)\n        \n        self.fc2 = nn.Linear(16, 128)\n        self.fc3 = nn.Linear(128, 100)\n        self.dropout = nn.Dropout(0.3)  # Adding dropout\n\n    def forward(self, x):\n        # Apply fully connected layer\n        x = torch.relu(self.fc1(x))\n        \n        # Reshape for 1D convolution\n        x = x.unsqueeze(1)  # Add a channel dimension\n        x = torch.relu(self.conv1d(x))\n        x = x.view(x.size(0), -1)  # Flatten\n        \n        # Apply linear layers\n        x = torch.relu(self.fc2(x))\n        x = torch.relu(self.fc3(x))\n        \n        # Apply dropout\n        x = self.dropout(x)\n        \n        return x'

In [18]:
def create_dataset(train_graphs, y, train_size, nb_test):
    yt = torch.tensor(y)[train_graphs]
    nb_train = train_size-nb_test
    nb_elt = nb_test*nb_train
    data = torch.empty((nb_elt, 2), dtype=torch.int)
    k = 0
    for test in train_graphs[-nb_test:]:
        data[k*nb_train:(k+1)*nb_train, 0] = test
        data[k*nb_train:(k+1)*nb_train, 1] = train_graphs[0:nb_train]
        k = k+1
    return data, yt


nb=len(Gs)


train_size=100
nb_test=10
train_graphs=torch.randperm(nb)[0:train_size]
print('train graphs:', train_graphs)



data,yt=create_dataset(train_graphs,y,train_size,nb_test)
print('data=',data,data.shape[0])
print('yt=',yt)


train graphs: tensor([ 57,  62,  36,  11, 114, 132, 153, 123,  79,  70,  19, 136, 137,  25,
          5, 138,  48,  91, 157, 116, 146, 129, 121, 147, 151,   0, 118,  46,
        133, 155,  44, 124,  39,  20,  73, 117, 149, 144, 162,  75,  74, 161,
         69,  55, 115,  63,  60, 100,  96,  42,   3,  23,   8, 127,  52, 112,
        152, 150,  14, 104,  66, 122,  15,  68,  51,  83,  56, 130,  37,  22,
        120,  93, 158,   4,  41,  64, 139,  33, 113,  16,  76,   1,  38, 119,
         82,  32, 103,  58,  84, 140, 143,  87,  81,  72,  97,  78,  43, 163,
        154,  31])
data= tensor([[143,  57],
        [143,  62],
        [143,  36],
        ...,
        [ 31,  58],
        [ 31,  84],
        [ 31, 140]], dtype=torch.int32) 900
yt= tensor([144.2000, 139.0000, 173.7000,  63.0000, 126.0000, 139.0000, 162.0000,
        177.0000,  91.5000, 175.7000, 166.0000, 173.0000, 159.5000,  59.5000,
         42.0000, 159.5000,  82.0000, 141.0000, 187.0000, 158.0000, 159.0000,
        151.5000, 14

In [19]:
""""import itertools
import random

# List of graphs
graph_list = Gs  # Replace with your actual graphs

# Labels for each graph
y = y  # Replace with your actual labels


# Shuffle the graph list and labels in the same order
combined = list(zip(graph_list, y))
random.shuffle(combined)
shuffled_graphs, shuffled_labels = zip(*combined)

# Split ratio for training and test datasets
split_ratio = 0.8
split_index = int(split_ratio * len(shuffled_graphs))
nb_test = len(Gs)-split_index


# Split the shuffled data into training and test datasets
train_graphs = shuffled_graphs[:split_index]
test_graphs = shuffled_graphs[split_index:]
train_labels = shuffled_labels[:split_index]
train_labels = torch.tensor(train_labels)
test_labels = shuffled_labels[split_index:]
test_labels = torch.tensor(test_labels)"""

""""import itertools

# Create pairs of graphs from train_graphs
train_graph_pairs = list(itertools.combinations(train_graphs, 2))
train_graph_pairs = from_networkx_to_tensor(train_graph_pairs)

test_graph_pairs = list(itertools.combinations(test_graphs, 2))
test_graph_pairs = from_networkx_to_tensor(test_graph_pairs)


"""




'"import itertools\n\n# Create pairs of graphs from train_graphs\ntrain_graph_pairs = list(itertools.combinations(train_graphs, 2))\ntrain_graph_pairs = from_networkx_to_tensor(train_graph_pairs)\n\ntest_graph_pairs = list(itertools.combinations(test_graphs, 2))\ntest_graph_pairs = from_networkx_to_tensor(test_graph_pairs)\n\n\n'

In [20]:

import torch.nn as nn
from collections import OrderedDict

net_instance = Net(Gs, normalize=False, node_label='extended_label')
regression_mlp_instance = RegressionMLP(input_size = len(net_instance.forward(data)))

model = nn.Sequential(OrderedDict([
    ('ComputeDist', net_instance),
    ('RegressFromDist', regression_mlp_instance)
]))


['C', 'O', 'S']
{'C': 0, 'O': 1, 'S': 2} 3
nb labels: 3  ; nb_edge_labels: 1
torch.Size([164, 121])
adjacency matrices tensor([[         0,          0,          1,  ..., -762590688,        611,
         -762590736],
        [         0,          0,          1,  ...,        611, -762379904,
                611],
        [         0,          0,          1,  ..., -762377024,        611,
         -762376976],
        ...,
        [         0,          0,          1,  ...,          1,          1,
                  0],
        [         0,          0,          0,  ...,          0,          0,
                  0],
        [         0,          0,          1,  ...,          0,          1,
                  0]], dtype=torch.int32)
node labels tensor([[        0,         0,         1,  ...,      3946,         0,
          41680896],
        [        0,         0,         2,  ...,         0,  92013181,
         292820609],
        [        0,         0,         2,  ...,         0, 343736320,
  

C:\Users\z004u7ws\AppData\Local\Temp\ipykernel_10324\1121286353.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return (A.view(1,A.shape[0]*A.shape[1]),torch.tensor(lab))


In [21]:
from collections import OrderedDict
from triangular_losses import TriangularConstraint as triangular_constraint
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from torch.optim.lr_scheduler import ReduceLROnPlateau

#regress.to(device)
def regression(model,data,yt,nb_iter):

    input=data

    target=yt
    
    criterion = torch.nn.MSELoss()
    criterionTri=triangular_constraint()
    
        
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=10, factor=0.1, verbose=True)

    print(device)
    
    InsDel=np.empty((nb_iter,2))
    node_costs,nodeInsDel,edge_costs,edgeInsDel=model[0].from_weighs_to_costs()
    nodeSub=np.empty((nb_iter,int(node_costs.shape[0]*(node_costs.shape[0]-1)/2)))
    edgeSub=np.empty((nb_iter,int(edge_costs.shape[0]*(edge_costs.shape[0]-1)/2)))
    loss_plt=np.empty(nb_iter)
    
    
    for t in range(nb_iter):    
        # Forward pass: Compute predicted y by passing data to the model    
        node_costs,nodeInsDel,edge_costs,edgeInsDel=model[0].from_weighs_to_costs()

        y_pred = model(input).to(device)
            
        # Compute and print loss
        loss = criterion(y_pred, target).to(device)                          
        #loss = criterion(y_pred, target).to(device)                          
        triangularInq=criterionTri(node_costs,nodeInsDel,edge_costs,edgeInsDel)
        
        loss=loss*(1+triangularInq)
        loss.to(device)
        InsDel[t][0]=nodeInsDel.item()
        InsDel[t][1]=edgeInsDel.item()
        rmse=np.sqrt(loss.item())
        loss_plt[t]=rmse
        print(t, rmse)   
        k=0
        for p in range(node_costs.shape[0]):
            for q in range(p+1,node_costs.shape[0]):
                nodeSub[t][k]=node_costs[p][q]
                k=k+1
        k=0
        for p in range(edge_costs.shape[0]):
            for q in range(p+1,edge_costs.shape[0]):
                edgeSub[t][k]=edge_costs[p][q]
                k=k+1
        if t % 1 == 0 or t==0:                         
            #print('Distances: ',y_pred)
            #print('alpha=',model[1].coef_dist*model[1].coef_dist)
            #print('lambda=',model[1].regu*model[1].regu)
            print('target=',target)
            print('y_pred=',y_pred)
            print('Loss Triangular:',triangularInq.item())
            print('node_costs :')
            print(node_costs)
            print('nodeInsDel:',nodeInsDel.item())
            print('edge_costs :')
            print(edge_costs)
            print('edgeInsDel:',edgeInsDel.item())
        
        
            # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        scheduler.step(loss)
        optimizer.step()
    
    return InsDel,nodeSub,edgeSub,loss_plt




In [22]:
print('Parameters:')
for param in model.parameters():
    print(type(param), param.size())
nb_iter= 50
InsDel, nodeSub,edgeSub,loss_plt=regression(model,data,yt,nb_iter)

Parameters:
<class 'torch.nn.parameter.Parameter'> torch.Size([4])
<class 'torch.nn.parameter.Parameter'> torch.Size([1])
<class 'torch.nn.parameter.Parameter'> torch.Size([256, 900])
<class 'torch.nn.parameter.Parameter'> torch.Size([256])
<class 'torch.nn.parameter.Parameter'> torch.Size([128, 256])
<class 'torch.nn.parameter.Parameter'> torch.Size([128])
<class 'torch.nn.parameter.Parameter'> torch.Size([100, 128])
<class 'torch.nn.parameter.Parameter'> torch.Size([100])
cpu
0 206.49983920998582
target= tensor([144.2000, 139.0000, 173.7000,  63.0000, 126.0000, 139.0000, 162.0000,
        177.0000,  91.5000, 175.7000, 166.0000, 173.0000, 159.5000,  59.5000,
         42.0000, 159.5000,  82.0000, 141.0000, 187.0000, 158.0000, 159.0000,
        151.5000, 148.5000, 195.0000, 186.5000, -23.7000, 188.9000,  69.0000,
        163.0000, 186.8000,  91.5000, 167.0000,  92.3000, 183.0000, 118.0000,
        147.0000, 250.0000, 166.5000, 215.0000, 107.0000, 117.1000, 228.0000,
        185.9000, 12

3 204.16193021716856
target= tensor([144.2000, 139.0000, 173.7000,  63.0000, 126.0000, 139.0000, 162.0000,
        177.0000,  91.5000, 175.7000, 166.0000, 173.0000, 159.5000,  59.5000,
         42.0000, 159.5000,  82.0000, 141.0000, 187.0000, 158.0000, 159.0000,
        151.5000, 148.5000, 195.0000, 186.5000, -23.7000, 188.9000,  69.0000,
        163.0000, 186.8000,  91.5000, 167.0000,  92.3000, 183.0000, 118.0000,
        147.0000, 250.0000, 166.5000, 215.0000, 107.0000, 117.1000, 228.0000,
        185.9000, 120.3000, 147.0000, 133.6000, 134.2000, 229.5000, 159.0000,
         82.0000,  10.8000,  52.5000, 148.5000, 201.0000, 112.0000, 128.5000,
        156.5000, 235.0000,  84.7000, 130.5000, 195.8000, 165.0000,  95.5000,
        181.0000,  96.0000,  90.0000, 145.0000, 165.5000, 181.0000,  63.6000,
        178.0000, 166.0000, 174.0000,  39.0000,  80.2000, 120.0000, 155.5000,
        112.5000, 109.5000,  92.0000, 102.5000,  37.3000,  99.5000, 170.0000,
         99.3000, 123.2000, 132.000

7 196.13691009088524
target= tensor([144.2000, 139.0000, 173.7000,  63.0000, 126.0000, 139.0000, 162.0000,
        177.0000,  91.5000, 175.7000, 166.0000, 173.0000, 159.5000,  59.5000,
         42.0000, 159.5000,  82.0000, 141.0000, 187.0000, 158.0000, 159.0000,
        151.5000, 148.5000, 195.0000, 186.5000, -23.7000, 188.9000,  69.0000,
        163.0000, 186.8000,  91.5000, 167.0000,  92.3000, 183.0000, 118.0000,
        147.0000, 250.0000, 166.5000, 215.0000, 107.0000, 117.1000, 228.0000,
        185.9000, 120.3000, 147.0000, 133.6000, 134.2000, 229.5000, 159.0000,
         82.0000,  10.8000,  52.5000, 148.5000, 201.0000, 112.0000, 128.5000,
        156.5000, 235.0000,  84.7000, 130.5000, 195.8000, 165.0000,  95.5000,
        181.0000,  96.0000,  90.0000, 145.0000, 165.5000, 181.0000,  63.6000,
        178.0000, 166.0000, 174.0000,  39.0000,  80.2000, 120.0000, 155.5000,
        112.5000, 109.5000,  92.0000, 102.5000,  37.3000,  99.5000, 170.0000,
         99.3000, 123.2000, 132.000

11 180.92612226748795
target= tensor([144.2000, 139.0000, 173.7000,  63.0000, 126.0000, 139.0000, 162.0000,
        177.0000,  91.5000, 175.7000, 166.0000, 173.0000, 159.5000,  59.5000,
         42.0000, 159.5000,  82.0000, 141.0000, 187.0000, 158.0000, 159.0000,
        151.5000, 148.5000, 195.0000, 186.5000, -23.7000, 188.9000,  69.0000,
        163.0000, 186.8000,  91.5000, 167.0000,  92.3000, 183.0000, 118.0000,
        147.0000, 250.0000, 166.5000, 215.0000, 107.0000, 117.1000, 228.0000,
        185.9000, 120.3000, 147.0000, 133.6000, 134.2000, 229.5000, 159.0000,
         82.0000,  10.8000,  52.5000, 148.5000, 201.0000, 112.0000, 128.5000,
        156.5000, 235.0000,  84.7000, 130.5000, 195.8000, 165.0000,  95.5000,
        181.0000,  96.0000,  90.0000, 145.0000, 165.5000, 181.0000,  63.6000,
        178.0000, 166.0000, 174.0000,  39.0000,  80.2000, 120.0000, 155.5000,
        112.5000, 109.5000,  92.0000, 102.5000,  37.3000,  99.5000, 170.0000,
         99.3000, 123.2000, 132.00

15 141.48411185094247
target= tensor([144.2000, 139.0000, 173.7000,  63.0000, 126.0000, 139.0000, 162.0000,
        177.0000,  91.5000, 175.7000, 166.0000, 173.0000, 159.5000,  59.5000,
         42.0000, 159.5000,  82.0000, 141.0000, 187.0000, 158.0000, 159.0000,
        151.5000, 148.5000, 195.0000, 186.5000, -23.7000, 188.9000,  69.0000,
        163.0000, 186.8000,  91.5000, 167.0000,  92.3000, 183.0000, 118.0000,
        147.0000, 250.0000, 166.5000, 215.0000, 107.0000, 117.1000, 228.0000,
        185.9000, 120.3000, 147.0000, 133.6000, 134.2000, 229.5000, 159.0000,
         82.0000,  10.8000,  52.5000, 148.5000, 201.0000, 112.0000, 128.5000,
        156.5000, 235.0000,  84.7000, 130.5000, 195.8000, 165.0000,  95.5000,
        181.0000,  96.0000,  90.0000, 145.0000, 165.5000, 181.0000,  63.6000,
        178.0000, 166.0000, 174.0000,  39.0000,  80.2000, 120.0000, 155.5000,
        112.5000, 109.5000,  92.0000, 102.5000,  37.3000,  99.5000, 170.0000,
         99.3000, 123.2000, 132.00

...

31 25.408635439212638
target= tensor([144.2000, 139.0000, 173.7000,  63.0000, 126.0000, 139.0000, 162.0000,
        177.0000,  91.5000, 175.7000, 166.0000, 173.0000, 159.5000,  59.5000,
         42.0000, 159.5000,  82.0000, 141.0000, 187.0000, 158.0000, 159.0000,
        151.5000, 148.5000, 195.0000, 186.5000, -23.7000, 188.9000,  69.0000,
        163.0000, 186.8000,  91.5000, 167.0000,  92.3000, 183.0000, 118.0000,
        147.0000, 250.0000, 166.5000, 215.0000, 107.0000, 117.1000, 228.0000,
        185.9000, 120.3000, 147.0000, 133.6000, 134.2000, 229.5000, 159.0000,
         82.0000,  10.8000,  52.5000, 148.5000, 201.0000, 112.0000, 128.5000,
        156.5000, 235.0000,  84.7000, 130.5000, 195.8000, 165.0000,  95.5000,
        181.0000,  96.0000,  90.0000, 145.0000, 165.5000, 181.0000,  63.6000,
        178.0000, 166.0000, 174.0000,  39.0000,  80.2000, 120.0000, 155.5000,
        112.5000, 109.5000,  92.0000, 102.5000,  37.3000,  99.5000, 170.0000,
         99.3000, 123.2000, 132.00

35 37.09158784241887
target= tensor([144.2000, 139.0000, 173.7000,  63.0000, 126.0000, 139.0000, 162.0000,
        177.0000,  91.5000, 175.7000, 166.0000, 173.0000, 159.5000,  59.5000,
         42.0000, 159.5000,  82.0000, 141.0000, 187.0000, 158.0000, 159.0000,
        151.5000, 148.5000, 195.0000, 186.5000, -23.7000, 188.9000,  69.0000,
        163.0000, 186.8000,  91.5000, 167.0000,  92.3000, 183.0000, 118.0000,
        147.0000, 250.0000, 166.5000, 215.0000, 107.0000, 117.1000, 228.0000,
        185.9000, 120.3000, 147.0000, 133.6000, 134.2000, 229.5000, 159.0000,
         82.0000,  10.8000,  52.5000, 148.5000, 201.0000, 112.0000, 128.5000,
        156.5000, 235.0000,  84.7000, 130.5000, 195.8000, 165.0000,  95.5000,
        181.0000,  96.0000,  90.0000, 145.0000, 165.5000, 181.0000,  63.6000,
        178.0000, 166.0000, 174.0000,  39.0000,  80.2000, 120.0000, 155.5000,
        112.5000, 109.5000,  92.0000, 102.5000,  37.3000,  99.5000, 170.0000,
         99.3000, 123.2000, 132.000

39 41.95455944940609
target= tensor([144.2000, 139.0000, 173.7000,  63.0000, 126.0000, 139.0000, 162.0000,
        177.0000,  91.5000, 175.7000, 166.0000, 173.0000, 159.5000,  59.5000,
         42.0000, 159.5000,  82.0000, 141.0000, 187.0000, 158.0000, 159.0000,
        151.5000, 148.5000, 195.0000, 186.5000, -23.7000, 188.9000,  69.0000,
        163.0000, 186.8000,  91.5000, 167.0000,  92.3000, 183.0000, 118.0000,
        147.0000, 250.0000, 166.5000, 215.0000, 107.0000, 117.1000, 228.0000,
        185.9000, 120.3000, 147.0000, 133.6000, 134.2000, 229.5000, 159.0000,
         82.0000,  10.8000,  52.5000, 148.5000, 201.0000, 112.0000, 128.5000,
        156.5000, 235.0000,  84.7000, 130.5000, 195.8000, 165.0000,  95.5000,
        181.0000,  96.0000,  90.0000, 145.0000, 165.5000, 181.0000,  63.6000,
        178.0000, 166.0000, 174.0000,  39.0000,  80.2000, 120.0000, 155.5000,
        112.5000, 109.5000,  92.0000, 102.5000,  37.3000,  99.5000, 170.0000,
         99.3000, 123.2000, 132.000

43 37.3780291806522
target= tensor([144.2000, 139.0000, 173.7000,  63.0000, 126.0000, 139.0000, 162.0000,
        177.0000,  91.5000, 175.7000, 166.0000, 173.0000, 159.5000,  59.5000,
         42.0000, 159.5000,  82.0000, 141.0000, 187.0000, 158.0000, 159.0000,
        151.5000, 148.5000, 195.0000, 186.5000, -23.7000, 188.9000,  69.0000,
        163.0000, 186.8000,  91.5000, 167.0000,  92.3000, 183.0000, 118.0000,
        147.0000, 250.0000, 166.5000, 215.0000, 107.0000, 117.1000, 228.0000,
        185.9000, 120.3000, 147.0000, 133.6000, 134.2000, 229.5000, 159.0000,
         82.0000,  10.8000,  52.5000, 148.5000, 201.0000, 112.0000, 128.5000,
        156.5000, 235.0000,  84.7000, 130.5000, 195.8000, 165.0000,  95.5000,
        181.0000,  96.0000,  90.0000, 145.0000, 165.5000, 181.0000,  63.6000,
        178.0000, 166.0000, 174.0000,  39.0000,  80.2000, 120.0000, 155.5000,
        112.5000, 109.5000,  92.0000, 102.5000,  37.3000,  99.5000, 170.0000,
         99.3000, 123.2000, 132.0000

47 26.862122187169753
target= tensor([144.2000, 139.0000, 173.7000,  63.0000, 126.0000, 139.0000, 162.0000,
        177.0000,  91.5000, 175.7000, 166.0000, 173.0000, 159.5000,  59.5000,
         42.0000, 159.5000,  82.0000, 141.0000, 187.0000, 158.0000, 159.0000,
        151.5000, 148.5000, 195.0000, 186.5000, -23.7000, 188.9000,  69.0000,
        163.0000, 186.8000,  91.5000, 167.0000,  92.3000, 183.0000, 118.0000,
        147.0000, 250.0000, 166.5000, 215.0000, 107.0000, 117.1000, 228.0000,
        185.9000, 120.3000, 147.0000, 133.6000, 134.2000, 229.5000, 159.0000,
         82.0000,  10.8000,  52.5000, 148.5000, 201.0000, 112.0000, 128.5000,
        156.5000, 235.0000,  84.7000, 130.5000, 195.8000, 165.0000,  95.5000,
        181.0000,  96.0000,  90.0000, 145.0000, 165.5000, 181.0000,  63.6000,
        178.0000, 166.0000, 174.0000,  39.0000,  80.2000, 120.0000, 155.5000,
        112.5000, 109.5000,  92.0000, 102.5000,  37.3000,  99.5000, 170.0000,
         99.3000, 123.2000, 132.00

In [23]:
num_fig=0
plt.figure(num_fig)
plt.plot(InsDel[:,0],label="node")
plt.plot(InsDel[:,1],label="edge")
plt.title('Node/Edge insertion/deletion costs')
num_fig=num_fig+1
plt.legend()
plt.figure(num_fig)
for k in range(nodeSub.shape[1]):
    plt.plot(nodeSub[:,k])
plt.title('Node Substitutions costs')
num_fig=num_fig+1
if edgeSub.shape[1] != 0:
    plt.figure(num_fig)
    for k in range(edgeSub.shape[1]):
        plt.plot(edgeSub[:,k])
    plt.title('Edge Substitutions costs')
    num_fig=num_fig+1
plt.figure(num_fig)
plt.plot(loss_plt[:])
plt.title('Loss')
plt.show()
plt.close()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Assuming you have your data already prepared in InsDel, nodeSub, edgeSub, and loss_plt

# Create a figure and axis
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
plt.subplots_adjust(wspace=0.3, hspace=0.3)

# Initialize empty line objects for animation
line1, = axs[0, 0].plot([], [], label="node")
line2, = axs[0, 0].plot([], [], label="edge")
axs[0, 0].set_title('Node/Edge insertion/deletion costs')
axs[0, 0].legend()

lines_node_sub = []
axs[0, 1].set_title('Node Substitutions costs')
for k in range(nodeSub.shape[1]):
    line, = axs[0, 1].plot([], [])
    lines_node_sub.append(line)

lines_edge_sub = []
if edgeSub.shape[1] != 0:
    axs[1, 0].set_title('Edge Substitutions costs')
    for k in range(edgeSub.shape[1]):
        line, = axs[1, 0].plot([], [])
        lines_edge_sub.append(line)

line_loss, = axs[1, 1].plot([], [])
axs[1, 1].set_title('Loss')

# Function to initialize the plot
def init():
    line1.set_data([], [])
    line2.set_data([], [])
    for line in lines_node_sub:
        line.set_data([], [])
    for line in lines_edge_sub:
        line.set_data([], [])
    line_loss.set_data([], [])
    return line1, line2, *lines_node_sub, *lines_edge_sub, line_loss

# Function to update the plot for each frame
def update(frame):
    x = range(len(InsDel))
    
    line1.set_data(x, InsDel[:, 0])
    line2.set_data(x, InsDel[:, 1])
    
    for k, line in enumerate(lines_node_sub):
        line.set_data(x, nodeSub[:, k])
    
    if edgeSub.shape[1] != 0:
        for k, line in enumerate(lines_edge_sub):
            line.set_data(x, edgeSub[:, k])
    
    line_loss.set_data(range(len(loss_plt)), loss_plt[:frame+1])
    
    return line1, line2, *lines_node_sub, *lines_edge_sub, line_loss

# Create the animation
ani = FuncAnimation(fig, update, frames=len(loss_plt), init_func=init, blit=True)

# Display the animation
plt.show()


In [ ]:
 y_pred_test = model(test_data)

In [ ]:
criterion = torch.nn.MSELoss()
criterionTri=triangular_constraint()
InsDel=np.empty((nb_iter,2))
node_costs,nodeInsDel,edge_costs,edgeInsDel=model[0].from_weighs_to_costs()
nodeSub=np.empty((nb_iter,int(node_costs.shape[0]*(node_costs.shape[0]-1)/2)))
edgeSub=np.empty((nb_iter,int(edge_costs.shape[0]*(edge_costs.shape[0]-1)/2)))
loss_plt=np.empty(nb_iter)
    
       


# Compute and print loss
loss = criterion(y_pred_test, test_yt)                         
#loss = criterion(y_pred, target).to(device)                          
triangularInq=criterionTri(node_costs,nodeInsDel,edge_costs,edgeInsDel)

loss=loss*(1+triangularInq)
loss.to(device)

rmse=np.sqrt(loss.item())


        


In [ ]:
y_pred_test

In [ ]:
test_yt

In [ ]:
g1 = A[:][0]
g2 = A[:][1]

In [ ]:
print(g2.shape)